In [1]:
#load libraries
from tensorflow.keras.datasets.cifar10 import load_data
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, LeakyReLU
from tensorflow.keras.layers import Conv2DTranspose, Reshape

In [2]:
(x_train,y_train),(x_test,y_test) = load_data()

In [3]:
#discriminator
def define_discriminator(input_shape = (32,32,3)):
    model = Sequential()
    #64 filters and each filter has a shape of 3*3
    model.add(Conv2D(64,(3,3),padding = "same", input_shape = input_shape ))
    model.add(LeakyReLU(alpha = 0.2))    #activation function

    #Pooling layer
    #downsampling  >>> reduces the resolution image, pooling, strides
    model.add(Conv2D(128,(3,3),padding = "same",strides = (2,2)))
    model.add(LeakyReLU(alpha = 0.2))
    #downsample
    model.add(Conv2D(128,(3,3),padding = "same",strides = (2,2)))
    model.add(LeakyReLU(alpha = 0.2))
    #downsample
    model.add(Conv2D(256,(3,3),padding = "same",strides = (2,2)))
    model.add(LeakyReLU(alpha = 0.2))

    #classification layers
    model.add(Flatten())
    model.add(Dropout(0.4))   #randomly
    model.add(Dense(1,activation = "sigmoid"))
    model.compile(loss = "binary_crossentropy",
                  optimizer = Adam(learning_rate = 0.0001),metrics = ["accuracy"])

    return model

In [4]:
model = define_discriminator()

/Users/mordhwaj/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
/Users/mordhwaj/anaconda3/lib/python3.11/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [5]:
#load the data
#and prepare training images
#x_train ,
def load_real_samples():
    (x_train,_),(_,_) = load_data()
    #int to float >>> better performance of algo with float data
    x = x_train.astype("float32")
    #scaling data  [0,255]    to  [-1,1]
    x = (x-127.5) / 127.5
    return x

In [6]:
x = load_real_samples()

In [7]:
#generate the real samples
#real->1 fake->0
def generate_real_samples(data, n):   #data = 100 n= 30
    #choose randomly the instances
    ri = np.random.randint(0,data.shape[0],n)
    x = data[ri]

    #generate class labels
    y = np.ones((n,1))
    return x, y

In [8]:
generate_real_samples

<function __main__.generate_real_samples(data, n)>

In [9]:
def generate_fake_samples(n):
    x = np.random.rand(32*32*3*n)
    #update to have a range [-1,1]
    x = -1 + x*2
    x = x.reshape((n,32,32,3))
    #class labels for fake data
    y = np.zeros((n,1))
    return x,y

In [10]:
x , y = generate_fake_samples(64)

In [11]:
def train_discriminator(model,dataset,n_iter = 30, batch_size = 128):
    half_batch = int(batch_size/2)
    for i in range(n_iter):
        x_real, y_real  = generate_real_samples(dataset,half_batch)
        #train on real samples>
        _,real_acc =model.train_on_batch(x_real,y_real)

        x_fake,y_fake =generate_fake_samples(half_batch)
        _,fake_acc =model.train_on_batch(x_fake,y_fake)

        print(" real = {} fake = {} ".format(real_acc*100,fake_acc*100))

In [12]:
model = define_discriminator()
dataset = load_real_samples()
train_discriminator(model,dataset)

 real = 32.8125 fake = 25.78125 
 real = 44.79166567325592 fake = 37.109375 
 real = 47.49999940395355 fake = 44.01041567325592 
 real = 50.66964030265808 fake = 49.21875 
 real = 53.81944179534912 fake = 53.75000238418579 
 real = 57.95454382896423 fake = 58.72395634651184 
 real = 61.77884340286255 fake = 62.94642686843872 
 real = 64.89583253860474 fake = 66.69921875 
 real = 68.47426295280457 fake = 69.96527910232544 
 real = 71.13487124443054 fake = 72.5781261920929 
 real = 73.21428656578064 fake = 74.36079382896423 
 real = 75.271737575531 fake = 76.30208134651184 
 real = 76.93750262260437 fake = 77.8245210647583 
 real = 78.53009104728699 fake = 79.296875 
 real = 79.79525923728943 fake = 80.46875 
 real = 81.09878897666931 fake = 81.689453125 
 real = 82.05492496490479 fake = 82.58271813392639 
 real = 82.85714387893677 fake = 83.33333134651184 
 real = 83.69932174682617 fake = 84.12829041481018 
 real = 84.41506624221802 fake = 84.80468988418579 
 real = 85.13719439506531 fa

In [14]:
#generator
def define_generator(latent_dim):
    model = Sequential()
    model.add(Dense(256*4*4,input_dim = latent_dim))
    model.add(LeakyReLU(alpha = 0.2))
    model.add((Reshape((4,4,256))))

    #upsampling 8*8
    model.add(Conv2DTranspose(128,(4,4),strides = (2,2),padding = "same"))
    model.add(LeakyReLU(alpha = 0.2))

     #upsampling 16*16
    model.add(Conv2DTranspose(128,(4,4),strides = (2,2),padding = "same"))
    model.add(LeakyReLU(alpha = 0.2))

     #upsampling 32*32
    model.add(Conv2DTranspose(128,(4,4),strides = (2,2),padding = "same"))
    model.add(LeakyReLU(alpha = 0.2))

    #output_layer
    model.add(Conv2D(3,(3,3),activation = "tanh",padding = "same"))

    return model

In [15]:
latent_dim = 100
model = define_generator(latent_dim)
model.summary()

/Users/mordhwaj/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 4096)           │       413,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_8 (LeakyReLU)       │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 8, 8, 128)      │       524,416 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_9 (LeakyReLU)       │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 16, 16, 128)    │       262,272 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_10 (LeakyReLU)      │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 32, 32, 128)    │       262,272 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_11 (LeakyReLU)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 32, 32, 3)      │         3,459 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,466,115 (5.59 MB)

 Trainable params: 1,466,115 (5.59 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
def generate_latent_points(latent_dim,n):  #n = rows latent_dim=columns (2,3) = 6
   x_input =  np.random.randn(latent_dim*n)
   x_input = x_input.reshape(n,latent_dim)
   return x_input

In [17]:
def generate_fake_samples(g_model,latent_dim,n):
    x_input = generate_latent_points(latent_dim,n)

    x = g_model.predict(x_input)

    y= np.zeros((n,1))
    return x,y

In [18]:
model = define_generator(latent_dim)
n_samples = 50
x,_ = generate_fake_samples(model,latent_dim,n_samples)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


In [19]:
#GANs
def define_gan(g_model,d_model):
    d_model.trainable = False
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    model.compile(loss = "binary_crossentropy",
                  optimizer  = Adam(learning_rate =0.0002 ))
    return model

In [20]:
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model,d_model)
gan_model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_5 (Sequential)       │ ?                      │     1,466,115 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ ?                      │       522,497 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,988,612 (7.59 MB)

 Trainable params: 1,466,115 (5.59 MB)

 Non-trainable params: 522,497 (1.99 MB)

In [21]:
def save_plot(samples,epoch , n=7):
    samples = (samples + 1 )/2.0
    for i in range(n*n):
        plt.subplot(n,n,1+i)
        plt.axis("off")
        plt.imshow(samples[i])
    filename = "generated_plot_%03d.png" % (epoch + 1)
    plt.savefig(filename)
    plt.close()

In [22]:
def summarise_performance(epoch,g_model,d_model,dataset,latent_dim,half_batch):   #10 number of epochs
    x_real,y_real = generate_real_samples(dataset,half_batch)
    _,real_acc=d_model.evaluate(x_real,y_real)
    x_fake,y_fake =generate_fake_samples(g_model,latent_dim,half_batch)#fake samples
    _,fake_acc=d_model.evaluate(x_fake,y_fake)
    print("Accuracy real : %.0f%% Fake : %0.f%%"% (real_acc*100,fake_acc*100) )
    save_plot(x_fake,epoch)
    filename = "generator_model%.3d.h5" % (epoch +1)  #0 +1 =001   HDF5 file format
    g_model.save(filename)

In [23]:
def train(g_model,d_model,gan_model,dataset,latent_dim,n_epochs=30,n_batch = 128):
    batch_per_epoch = int(dataset.shape[0]/n_batch)
    half_batch = int(n_batch/2)
    for i in range(n_epochs):
        for j in range(batch_per_epoch):
            #fetching real data
            x_real, y_real  = generate_real_samples(dataset,half_batch)
            d_loss1, _ =d_model.train_on_batch(x_real,y_real)


            x_fake,y_fake =generate_fake_samples(g_model,latent_dim,half_batch)#fake samples
            d_loss2,_ = d_model.train_on_batch(x_fake,y_fake)
            x_gan = generate_latent_points(latent_dim,n_batch)  #vector space
            y_gan = np.ones((n_batch,1))
            gan_loss = gan_model.train_on_batch(x_gan,y_gan)
            print(f"epoch: {i+1}/{n_epochs} d1(real): {d_loss1 } d2(fake):{d_loss2}")

            if (i+1) %10 ==0:
                summarise_performance(i,g_model,d_model,dataset,latent_dim,half_batch)


In [24]:
train(g_model,d_model,gan_model,dataset,latent_dim)

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step

/Users/mordhwaj/anaconda3/lib/python3.11/site-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
epoch: 1/2 d1(real): 0.7002760171890259 d2(fake):0.6967867612838745
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
epoch: 1/2 d1(real): 0.6985089182853699 d2(fake):0.6974316835403442
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
epoch: 1/2 d1(real): 0.6978170275688171 d2(fake):0.6973891854286194
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
epoch: 1/2 d1(real): 0.6983045935630798 d2(fake):0.6981022953987122
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
epoch: 1/2 d1(real): 0.6981958150863647 d2(fake):0.6982124447822571
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
epoch: 1/2 d1(real): 0.6984850764274597 d2(fake):0.6986647248268127
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
epoch: 1/2 d1(real): 0.6986426115036011 d2(fake):0.6989652514457703
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
epoch: 1/2 d1(real): 0.6993232369422913 d2(fake):0.6997926831245422
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
epoch: 1/2 d1(real): 0.6995057463645935 d2(fake):0.700175404548645
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
epoch: 1/

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
epoch: 1/2 d1(real): 0.7922177910804749 d2(fake):0.7946322560310364
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
epoch: 1/2 d1(real): 0.7938383221626282 d2(fake):0.7962780594825745
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
epoch: 1/2 d1(real): 0.7954811453819275 d2(fake):0.7979356050491333
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
epoch: 1/2 d1(real): 0.7971492409706116 d2(fake):0.7994879484176636
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
epoch: 1/2 d1(real): 0.7986982464790344 d2(fake):0.8010730743408203
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
epoch: 1/2 d1(real): 0.800304114818573 d2(fake):0.8027267456054688
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
epoch: 1/2 d1(real): 0.8019271492958069 d2(fake):0.8042776584625244
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
epoch: 1/2 d1(real): 0.8034707903862 d2(fake):0.8057987093925476
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
epoch: 1/2 d1(real): 0.8050075173377991 d2(fake):0.8073503971099854
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
epoch: 1/2 d

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
epoch: 1/2 d1(real): 0.8656375408172607 d2(fake):0.8671849370002747
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
epoch: 1/2 d1(real): 0.8664931058883667 d2(fake):0.8680662512779236
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
epoch: 1/2 d1(real): 0.8673677444458008 d2(fake):0.8689595460891724
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
epoch: 1/2 d1(real): 0.8682607412338257 d2(fake):0.8698430061340332
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
epoch: 1/2 d1(real): 0.8691486716270447 d2(fake):0.8706632852554321
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
epoch: 1/2 d1(real): 0.8699702620506287 d2(fake):0.8714523315429688
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
epoch: 1/2 d1(real): 0.8707616329193115 d2(fake):0.872253954410553
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
epoch: 1/2 d1(real): 0.8715588450431824 d2(fake):0.8730659484863281
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
epoch: 1/2 d1(real): 0.8723800182342529 d2(fake):0.8738707900047302
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
epoch: 1/

...

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
epoch: 2/2 d1(real): 1.1525006294250488 d2(fake):1.2086442708969116
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.1562 - loss: 0.7001
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0000e+00 - loss: 1.3774
Accuracy real : 16% Fake : 0%


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
epoch: 2/2 d1(real): 1.1521371603012085 d2(fake):1.2086423635482788
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2812 - loss: 0.7002
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0000e+00 - loss: 1.3784
Accuracy real : 27% Fake : 0%


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
epoch: 2/2 d1(real): 1.1537920236587524 d2(fake):1.209916114807129
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3125 - loss: 0.6970
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0000e+00 - loss: 1.3791
Accuracy real : 33% Fake : 0%


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
epoch: 2/2 d1(real): 1.15259850025177 d2(fake):1.2116066217422485
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2292 - loss: 0.7006
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0000e+00 - loss: 1.3781
Accuracy real : 23% Fake : 0%


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
epoch: 2/2 d1(real): 1.1526998281478882 d2(fake):1.2104004621505737
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2917 - loss: 0.6996
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0000e+00 - loss: 1.3790
Accuracy real : 25% Fake : 0%
